In [9]:
# POLIGONS for KRAJE, OKRESY, OBCE
# -*- coding: utf-8 -*-
import json
from shapely.geometry import shape, Point


# # Load your kraje polygons from JSON
# with open('C://Users//jirip//Documents//Developer//python//kriminalita//map_files//kraje.json') as f:
#     regions_data = json.load(f)
# # Prepare polygons
# regions = []
# for feature in regions_data['features']:  # assuming GeoJSON-like structure
#     region_id = feature['id']
#     polygon = shape(feature['geometry'])  # shapely geometry from geojson
#     regions.append((region_id, polygon))


# Load your okresy polygons from JSON
with open('C://Users//jirip//Documents//Developer//python//kriminalita//map_files//okresy.json') as f:
    districts_data = json.load(f)
# Prepare polygons
districts = []
for feature in districts_data['features']:  # assuming GeoJSON-like structure
    district_id = feature['id']
    polygon = shape(feature['geometry'])  # shapely geometry from geojson
    districts.append((district_id, polygon))


#  Load your obce polygons from JSON
with open('C://Users//jirip//Documents//Developer//python//kriminalita//map_files//obce.json') as f:
    cities_data = json.load(f)
# Prepare polygons
cities = []
for feature in cities_data['features']:  # assuming GeoJSON-like structure
    city_id = feature['id']
    polygon = shape(feature['geometry'])  # shapely geometry from geojson
    cities.append((city_id, polygon))

In [13]:
# POLIGONS for OBCE
# -*- coding: utf-8 -*-
import json
from shapely.geometry import shape, Point
# Load obce polygons from JSON
with open('C://Users//jirip//Documents//Developer//python//kriminalita//map_files//okresy.json') as f:
    regions_data = json.load(f)

# Prepare polygons
regions = []
for feature in regions_data['features']:  # assuming GeoJSON-like structure
    region_id = feature['id']
    polygon = shape(feature['geometry'])  # shapely geometry from geojson
    regions.append((region_id, polygon))

# Example point you want to assign
point = Point(13.295537,49.387969)  # (longitude, latitude)

# Find which region contains the point
assigned_region = None
for region_id, polygon in regions:
    if polygon.contains(point):
        assigned_region = region_id
        break

if assigned_region:
    print(f"Point is inside region: {assigned_region}")
else:
    print("Point is not inside any region.")


Point is inside region: CZ0322000000


In [ ]:
# FLATTENING GEOJSON
# -*- coding: utf-8 -*-
import json
import pandas as pd


# Open and load the JSON file
with open('C://Users//jirip//Documents//Developer//python//kriminalita//202504.geojson', 'r') as file:
    data = json.load(file)


rows = []
for feature in data['features']:
    properties = feature['properties']
    geometry = feature['geometry']
    row = properties.copy()  # Start with properties
    row['geometry_type'] = geometry['type']
    row['longitude'] = geometry['coordinates']
    row['latitude'] = geometry['coordinates'][1]
    rows.append(row)

df = pd.DataFrame(rows)
print(df)


In [ ]:
# POLIGONS for OBCE & ASSIGNING POINTS TO POLYGONS
# -*- coding: utf-8 -*-
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, Point

# Load obce polygons from JSON
with open('C://Users//jirip//Documents//Developer//python//kriminalita//map_files//okresy.json') as f:
    regions_data = json.load(f)

# Prepare polygons
regions = []
for feature in regions_data['features']:  # assuming GeoJSON-like structure
    region_id = feature['id']
    polygon = shape(feature['geometry'])  # shapely geometry from geojson
    regions.append((region_id, polygon))



#  Load your obce polygons from JSON
with open('C://Users//jirip//Documents//Developer//python//kriminalita//map_files//obce.json') as f:
    cities_data = json.load(f)
# Prepare polygons
cities = []
for feature in cities_data['features']:  # assuming GeoJSON-like structure
    city_id = feature['id']
    polygon = shape(feature['geometry'])  # shapely geometry from geojson
    cities.append((city_id, polygon))



# yearMonths = ['202501','202502','202503','202504','202401','202402','202403','202404','202405','202406','202407','202408','202409','202410','202411','202412','202301','202302','202303','202304','202305','202306','202307','202308','202309','202310','202311','202312','202201','202202','202203','202204','202205','202206','202207','202208','202209','202210','202211','202212','202101','202102','202103','202104','202105','202106','202107','202108','202109','202110','202111','202112','202001','202002','202003','202004','202005','202006','202007','202008','202009','202010','202011','202012','201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912','201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812','201701','201702','201703','201704','201705','201706','201707','201708','201709','201710','201711','201712','201601','201602','201603','201604','201605','201606','201607','201608','201609','201610','201611','201612','201501','201502','201503','201504','201505','201506','201507','201508','201509','201510','201511','201512','201401','201402','201403','201404','201405','201406','201407','201408','201409','201410','201411','201412','201301','201302','201303','201304','201305','201306','201307','201308','201309','201310','201311','201312','201201','201202','201203','201204','201205','201206','201207','201208','201209','201210','201211','201212']
yearMonths = ['202501', '202502']

for yearMonth in yearMonths:
    df = pd.read_csv(
        f"C://Users//jirip//Documents//Developer//python//kriminalita//processed_files//fact_clean_{yearMonth}.csv", sep=",", encoding="utf-8")

    # Prepare regions GeoDataFrame
    gdf_regions = gpd.GeoDataFrame(
        {'region_id': [r[0] for r in regions]}, geometry=[r[1] for r in regions])

    # Prepare points GeoDataFrame
    gdf_points = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

    # Spatial join
    gdf_joined = gpd.sjoin(gdf_points, gdf_regions,how='left', predicate='within')

    # If needed, bring the result back to pandas
    df_result = pd.DataFrame(gdf_joined.drop(columns=['geometry', 'index_right']))
    # df_result['mp'] = df_result['mp'].astype(int)
    # df_result['region_id'] = df_result['region_id'].str[:-6]

    # Prepare cities GeoDataFrame
    gdf_cities = gpd.GeoDataFrame({'city_id': [r[0] for r in cities]}, geometry=[r[1] for r in cities])

    # Prepare points GeoDataFrame
    gdf_points = gpd.GeoDataFrame(df_result, geometry=gpd.points_from_xy(df_result.Longitude, df_result.Latitude))

    # Spatial join
    gdf_joined = gpd.sjoin(gdf_points, gdf_cities,how='left', predicate='within')

    # If needed, bring the result back to pandas
    df_result_final = pd.DataFrame(gdf_joined.drop(columns=['geometry', 'index_right', 'Longitude', 'Latitude']))
    df_result_final['mp'] = df_result_final['mp'].astype(int)
    df_result_final['city_id'] = df_result_final['city_id'].fillna(df_result_final['region_id'])
    df_result_final = df_result_final.drop(columns=['region_id'])
    df_result_final.to_csv(f'F_Crime_{yearMonth}.csv', index=False)

In [ ]:
# CREATE D_LOCATION CSV
# -*- coding: utf-8 -*-

import pandas as pd
import json
# Load the JSON file into a DataFrame
with open('C://Users//jirip//Documents//Developer//python//kriminalita//obce.json', 'r') as file:
    obce_data = json.load(file)

# Extract relevant data into a DataFrame
obce_data = pd.json_normalize(obce_data['features'])
obce_data = obce_data[['id', 'name']]
print(obce_data.head())
obce_data.to_csv('D_location.csv', index=False)


In [ ]:
# DONWLOAD ALL DATA FROM URLS
import requests
import zipfile
import io

# dates = [202501,202502,202503,202504,'202401','202402','202403','202404','202405','202406','202407','202408','202409','202410','202411','202412','202301','202302','202303','202304','202305','202306','202307','202308','202309','202310','202311','202312','202201','202202','202203','202204','202205','202206','202207','202208','202209','202210','202211','202212','202101','202102','202103','202104','202105','202106','202107','202108','202109','202110','202111','202112','202001','202002','202003','202004','202005','202006','202007','202008','202009','202010','202011','202012','201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912','201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812','201701','201702','201703','201704','201705','201706','201707','201708','201709','201710','201711','201712','201601','201602','201603','201604','201605','201606','201607','201608','201609','201610','201611','201612','201501','201502','201503','201504','201505','201506','201507','201508','201509','201510','201511','201512','201401','201402','201403','201404','201405','201406','201407','201408','201409','201410','201411','201412','201301','201302','201303','201304','201305','201306','201307','201308','201309','201310','201311','201312','201201','201202','201203','201204','201205','201206','201207','201208','201209','201210','201211','201212']
 

for date in dates:
    # URL of the zip file
    url = f'https://kriminalita.policie.gov.cz/api/v2/downloads/{date}.zip'

    # Download the zip file
    response = requests.get(url)
    if response.status_code == 200:
        # Extract the zip file
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            z.extractall('path_to_extract')  # Replace 'path_to_extract' with your desired directory
        print(f"Download and extraction complete for {date}.")
    else:
        print(f"Failed to download file for {date}. Status code: {response.status_code}")


In [25]:
# COMBINE CSV FILES
# -*- coding: utf-8 -*-
import os
import pandas as pd

# Path to the folder containing the CSV files
folder_path = 'mapped_files'  
# List to store DataFrames
dataframes = []

# Iterate through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):  # Check if the file is a CSV
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)  # Read the CSV into a DataFrame
        dataframes.append(df)  # Append the DataFrame to the list

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

print(len(combined_df))  # Display the first few rows of the combined DataFrame

18433166
